In [ ]:
%pip install langchain langchain-community langchain-openai python-dotenv pyvis neo4j 

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()  # loads .env into environment

True

In [14]:
from langchain_community.graphs import Neo4jGraph
import os

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)


In [15]:
graph.query("RETURN 1")

[{'1': 1}]

In [16]:
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document

llm = ChatOpenAI(
    model="openai/gpt-4.1-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://models.github.ai/inference"
)

transformer = LLMGraphTransformer(llm=llm)

text = """
Albert Einstein[a] (14 March 1879 – 18 April 1955) was a German-born theoretical physicist best known for developing the theory of relativity. Einstein also made important contributions to quantum theory.[1][5] His mass–energy equivalence formula E = mc2, which arises from special relativity, has been called "the world's most famous equation".[6] He received the 1921 Nobel Prize in Physics for "his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect".[7]

Born in the German Empire, Einstein moved to Switzerland in 1895, forsaking his German citizenship (as a subject of the Kingdom of Württemberg)[note 1] the following year. In 1897, at the age of seventeen, he enrolled in the mathematics and physics teaching diploma program at the Swiss federal polytechnic school in Zurich, graduating in 1900. He acquired Swiss citizenship a year later, which he kept for the rest of his life, and afterwards secured a permanent position at the Swiss Patent Office in Bern. In 1905, he submitted a successful PhD dissertation to the University of Zurich. In 1914, he moved to Berlin to join the Prussian Academy of Sciences and the Humboldt University of Berlin, becoming director of the Kaiser Wilhelm Institute for Physics in 1917; he also became a German citizen again, this time as a subject of the Kingdom of Prussia.[note 1] In 1933, while Einstein was visiting the United States, Adolf Hitler came to power in Germany. Horrified by the Nazi persecution of his fellow Jews,[8] he decided to remain in the US, and was granted American citizenship in 1940.[9] On the eve of World War II, he endorsed a letter to President Franklin D. Roosevelt alerting him to the potential German nuclear weapons program and recommending that the US begin similar research, later carried out as the Manhattan Project.

In 1905, sometimes described as his annus mirabilis (miracle year), he published four groundbreaking papers.[10] In them, he outlined a theory of the photoelectric effect, explained Brownian motion, introduced his special theory of relativity, and demonstrated that if the special theory is correct, mass and energy are equivalent to each other. In 1915, he proposed a general theory of relativity that extended his system of mechanics to incorporate gravitation. A paper that he published the following year laid out the implications of general relativity for the modeling of the structure and evolution of the universe as a whole.[11][12] It introduced the cosmological constant and is further regarded as the first step in the field of modern theoretical cosmology. In 1917, Einstein wrote a paper which introduced the concepts of spontaneous emission and stimulated emission, the latter of which is the core mechanism behind the laser and maser, and which contained a trove of information that would be beneficial to developments in physics later on, such as quantum electrodynamics and quantum optics.[13]

In the middle part of his career, Einstein made important contributions to statistical mechanics and quantum theory. Especially notable was his work on the quantum physics of radiation, in which light consists of particles, subsequently called photons. With physicist Satyendra Nath Bose, he laid the groundwork for Bose–Einstein statistics. For much of the last phase of his academic life, Einstein worked on two endeavors that ultimately proved unsuccessful. First, he advocated against quantum theory's introduction of fundamental randomness into science's picture of the world, objecting that "God does not play dice".[14] Second, he attempted to devise a unified field theory by generalizing his geometric theory of gravitation to include electromagnetism. As a result, he became increasingly isolated from mainstream modern physics. Many things are named after him, including the element Einsteinium. In 1999, he was named Time's Person of the Century.[15]
"""

documents = [Document(page_content=text)]

graph_documents = transformer.convert_to_graph_documents(documents)


In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='Albert Einstein', type='Person', properties={}), Node(id='Theory Of Relativity', type='Concept', properties={}), Node(id='Quantum Theory', type='Concept', properties={}), Node(id='E = Mc2', type='Concept', properties={}), Node(id='1921 Nobel Prize In Physics', type='Event', properties={}), Node(id='Photoelectric Effect', type='Concept', properties={}), Node(id='German Empire', type='Location', properties={}), Node(id='Switzerland', type='Location', properties={}), Node(id='Kingdom Of Württemberg', type='Location', properties={}), Node(id='Swiss Federal Polytechnic School In Zurich', type='Organization', properties={}), Node(id='University Of Zurich', type='Organization', properties={}), Node(id='Swiss Patent Office In Bern', type='Organization', properties={}), Node(id='Berlin', type='Location', properties={}), Node(id='Prussian Academy Of Sciences', type='Organization', properties={}), Node(id='Humboldt University Of Berlin', type='Organization', propert

In [18]:
graph.add_graph_documents(graph_documents)

In [19]:
result = graph.query("""
MATCH (a)-[r]->(b)
RETURN a.id, type(r), b.id
LIMIT 10
""")

print(result)


[{'a.id': 'Neo4J', 'type(r)': 'IS_A', 'b.id': 'Graph Database'}, {'a.id': 'Neo4J', 'type(r)': 'USED_FOR', 'b.id': 'Knowledge Graph Storage'}, {'a.id': 'Langchain', 'type(r)': 'IS_A', 'b.id': 'Framework For Building Llm Applications'}, {'a.id': 'Langchain', 'type(r)': 'CAN_INTEGRATE_WITH', 'b.id': 'Neo4J'}, {'a.id': 'Albert Einstein', 'type(r)': 'KNOWN_FOR', 'b.id': 'Theory Of Relativity'}, {'a.id': 'Albert Einstein', 'type(r)': 'CONTRIBUTED_TO', 'b.id': 'Quantum Theory'}, {'a.id': 'Albert Einstein', 'type(r)': 'CONTRIBUTED_TO', 'b.id': 'Modern Theoretical Cosmology'}, {'a.id': 'Albert Einstein', 'type(r)': 'CONTRIBUTED_TO', 'b.id': 'Quantum Electrodynamics'}, {'a.id': 'Albert Einstein', 'type(r)': 'CONTRIBUTED_TO', 'b.id': 'Quantum Optics'}, {'a.id': 'Albert Einstein', 'type(r)': 'CONTRIBUTED_TO', 'b.id': 'Statistical Mechanics'}]
